# Installation des différents package

In [1]:
!pip install semantic-kernel python-dotenv

# Import des variables d'environnement

N'oubliez pas de copier-coller le fichier .env à la racine du code

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# Import des librairies nécessaires

In [3]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
import logging
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
import os
from semantic_kernel.functions import kernel_function
from datetime import datetime
import requests

# Définition du Tool permettant de récupérer la météo

In [4]:
class WeatherPlugin:
    """Weather plugin for Semantic Kernel with Open-Meteo API integration."""

    @kernel_function(
    name="get_current_weather",
    description="Fetches current weather for given coordinates using Open-Meteo API"
    )
    def get_current_weather(
        self,
        latitude: float,
        longitude: float
    ) -> dict:
        """Fetches current weather for given coordinates.

        Args:
            latitude: Latitude coordinate
            longitude: Longitude coordinate

        Returns:
            dict: Weather details with status
        """
        url = "https://api.open-meteo.com/v1/forecast"

        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current": [
                "temperature_2m",
                "relative_humidity_2m",
                "precipitation",
                "weather_code",
                "wind_speed_10m",
            ],
            "timezone": "auto",
        }

        try:
            response = requests.get(url, params=params)

            if response.status_code != 200:
                return {"status": "error", "error_message": f"API Error: {response.text}"}

            data = response.json()
            current = data["current"]

            weather_description = self._get_weather_description(current["weather_code"])

            return {
                "status": "success",
                "latitude": data["latitude"],
                "longitude": data["longitude"],
                "description": weather_description,
                "temperature": current["temperature_2m"],
                "humidity": current["relative_humidity_2m"],
                "wind_speed": current["wind_speed_10m"],
                "precipitation_1h": current["precipitation"],
            }
        except Exception as e:
            return {"status": "error", "error_message": f"Request failed: {str(e)}"}

    @kernel_function(
        name="get_weather_forecast",
        description="Fetches daily weather forecast for up to 16 days using Open-Meteo API"
    )
    def get_weather_forecast(
        self,
        latitude: float,
        longitude: float,
        days: int = 7
    ) -> dict:
        """Fetches daily weather forecast.

        Args:
            latitude: Latitude coordinate
            longitude: Longitude coordinate
            days: Number of days for forecast (default 7, max 16)

        Returns:
            dict: Forecast details with status
        """
        url = "https://api.open-meteo.com/v1/forecast"

        days = min(days, 16)

        params = {
            "latitude": latitude,
            "longitude": longitude,
            "daily": [
                "temperature_2m_max",
                "temperature_2m_min",
                "weather_code",
                "precipitation_sum",
                "wind_speed_10m_max",
                "wind_gusts_10m_max",
                "wind_direction_10m_dominant",
            ],
            "timezone": "auto",
            "forecast_days": days,
        }

        try:
            response = requests.get(url, params=params)

            if response.status_code != 200:
                return {"status": "error", "error_message": f"API Error: {response.text}"}

            data = response.json()
            daily = data["daily"]

            forecasts = []
            for i in range(len(daily["time"])):
                weather_description = self._get_weather_description(daily["weather_code"][i])
                forecasts.append(
                    {
                        "date": daily["time"][i],
                        "temp_max": daily["temperature_2m_max"][i],
                        "temp_min": daily["temperature_2m_min"][i],
                        "description": weather_description,
                        "precipitation": daily["precipitation_sum"][i],
                        "wind_speed_max": daily["wind_speed_10m_max"][i],
                        "wind_gusts_max": daily["wind_gusts_10m_max"][i],
                        "wind_direction": daily["wind_direction_10m_dominant"][i],
                    }
                )

            return {
                "status": "success",
                "latitude": data["latitude"],
                "longitude": data["longitude"],
                "forecasts": forecasts,
            }
        except Exception as e:
            return {"status": "error", "error_message": f"Request failed: {str(e)}"}

    @kernel_function(
        name="get_current_date",
        description="Fetches the current date and time"
    )
    def get_current_date(self) -> dict:
        """Fetches the current date and time.

        Returns:
            dict: Current date and time with status
        """
        try:
            current_date_time = datetime.utcnow().isoformat()
            return {"status": "success", "current_date_time": current_date_time}
        except Exception as e:
            return {"status": "error", "error_message": str(e)}

    def _get_weather_description(self, code: int) -> str:
        """Maps WMO weather code to description.

        Args:
            code: WMO weather code

        Returns:
            str: Weather description
        """
        weather_codes = {
            0: "clear sky",
            1: "mainly clear",
            2: "partly cloudy",
            3: "overcast",
            45: "fog",
            48: "depositing rime fog",
            51: "light drizzle",
            53: "moderate drizzle",
            55: "dense drizzle",
            56: "light freezing drizzle",
            57: "dense freezing drizzle",
            61: "slight rain",
            63: "moderate rain",
            65: "heavy rain",
            66: "light freezing rain",
            67: "heavy freezing rain",
            71: "slight snow",
            73: "moderate snow",
            75: "heavy snow",
            77: "snow grains",
            80: "slight rain showers",
            81: "moderate rain showers",
            82: "violent rain showers",
            85: "slight snow showers",
            86: "heavy snow showers",
            95: "thunderstorm",
            96: "thunderstorm with slight hail",
            99: "thunderstorm with heavy hail",
        }
        return weather_codes.get(code, "unknown")


In [ ]:

async def main():
    # Initialize the kernel
    kernel = Kernel()

    # Add Azure OpenAI chat completion
    chat_completion = AzureChatCompletion(
        deployment_name=os.getenv("DEPLOYMENT_NAME"),
        api_key=os.getenv("API_KEY"),
        base_url=os.getenv("BASE_URL"),
    )
    kernel.add_service(chat_completion)

    # Set the logging level for  semantic_kernel.kernel to DEBUG.
    setup_logging()
    logging.getLogger("kernel").setLevel(logging.DEBUG)

    # Add a plugin (the LightsPlugin class is defined below)
    # kernel.add_plugin(
    #     LightsPlugin(),
    #     plugin_name="Lights",
    # )

    # Enable planning
    execution_settings = AzureChatPromptExecutionSettings()
    execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    # Create a history of the conversation
    history = ChatHistory()

    # Initiate a back-and-forth chat
    userInput = None
    while True:
        # Collect user input
        userInput = input("User > ")

        # Terminate the loop if the user says "exit"
        if userInput == "exit":
            break

        # Add user input to the history
        history.add_user_message(userInput)

        # Get the response from the AI
        result = await chat_completion.get_chat_message_content(
            chat_history=history,
            settings=execution_settings,
            kernel=kernel,
        )

        # Print the results
        print("Assistant > " + str(result))

        # Add the message from the agent to the chat history
        history.add_message(result)

await main()